In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root_body(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),#undef)

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("b2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
x = MechanismState{Float64}(mechanism);

In [6]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  RigidBodyDynamics.Joint… => [0.7306928931363796,0.7848672038378661,0.14700280…
  RigidBodyDynamics.Joint… => [0.05925801585523671]

In [7]:
num_positions(x)

8

In [8]:
num_velocities(x)

7

In [9]:
cache = MechanismStateCache(mechanism, x);

RigidBodyDynamics.CartesianFrame3D("world")
RigidBodyDynamics.CartesianFrame3D("after_1")


In [10]:
relative_transform(cache, joint2.frameAfter, body1.frame)

RigidBodyDynamics.Transform3D{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("body1"),-0.2812998862624654 + 0.08232380006473256im + 0.9343074583220061jm + 0.20288602528389768km,Vec(0.6899115564849716,0.272084151458476,0.6082573818677589))

In [11]:
center_of_mass(cache)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(1.2248955179124694,0.36493535073200234,0.8054958742094717))

In [12]:
mass(mechanism)

1.4756578583275417

In [13]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(1.625301683095627,0.016147740140022543,0.7649497396076184))

In [14]:
relative_twist(cache, body2, body1)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(-0.026575475627624856,0.015366169853575392,0.001866604773344327),Vec(-0.011724206179434815,-0.023362699041091006,0.025403795600281776))

In [15]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.0,0.0,0.0),Vec(0.0,0.0,0.0))

In [16]:
geometric_jacobian(cache, path(mechanism, body2, world))

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
  0.448471   -0.381886  -0.723763   …  -0.0       -0.0       -0.0     
 -0.25931    -0.588421   0.669953      -0.0       -0.0       -0.0     
 -0.0314996  -0.712688  -0.165318      -0.0       -0.0       -0.0     
  0.19785    -0.270755   0.0603874     -0.381886  -0.723763  -0.574744
  0.394254    0.736703   0.267918      -0.588421   0.669953  -0.452685
 -0.428698   -0.463168   0.821368   …  -0.712688  -0.165318   0.681723)

In [17]:
mass_matrix(cache)

7x7 Array{Float64,2}:
  0.35349     0.00236167  -1.29167    1.48672   …  -0.97525      -0.0908289
 -0.331202    0.91297      3.17484   -0.218282     -0.413966     -0.200257 
  0.210991   -1.5709      -0.829279  -0.223426      1.57725       0.194262 
 -0.184023   -1.02889     -1.88107    0.0           0.540363      0.252786 
 -0.38254     0.276359    -0.245755   0.553095     -1.27302      -0.0328033
 -1.30978     0.308812     1.93687   -0.540363  …   2.77556e-17   0.0303056
 -0.0908289  -0.200257     0.194262   0.252786      0.0303056    -0.0245132

In [18]:
A = momentum_matrix(cache)

RigidBodyDynamics.MomentumMatrix{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
  0.125853   -0.568528    0.632313   …   0.576307   0.457648   0.0162691 
  0.0285512  -0.46416    -0.724048      -0.53346    0.360457   0.00665842
 -0.09101     0.608008    0.052298       0.131637  -0.542832   0.176815  
 -1.09993    -0.0154135   0.216986       0.520444  -0.715012  -0.189637  
 -0.444918   -0.650776   -0.0654222      0.250826   1.17083    0.0880427 
  0.698518   -0.898117   -2.70165    …  -1.26203    0.174656  -0.367903  )

In [19]:
A * velocity_vector(x)

RigidBodyDynamics.Momentum{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.38799494306485066,-0.6205666152710565,0.38114484198627324),Vec(-0.7002449641611648,-0.2641906416814828,-1.3892488245443542))

In [ ]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(cache, accel, world.frame)

RigidBodyDynamics.SpatialAcceleration{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(-0.8148494409858584,-0.041556086910988665,-0.5330777686243846),Vec(-0.081408832385965,0.317690488185878,-0.10391521269933167))

In [ ]:
bias_acceleration(joint1, x.q[joint1], x.v[joint1])